# Rocket science challenge, datahack 2017, Klil group
## Approach
We spent most of the competition time on a failing attempt to cleverly split the data to segments of 5 seconds, in order to "augment" the amount of samples and use all the available data.
On hindsight, in the given timeframe we should have just split the data to "atoms" - each sample is a bunch of non sequential observations, or use a model that can understand "time", such as LSTM.

Anyhow, we do have a noteworthy feature here, which is the drag force. We see that the drag in the X direction is much more important than the drag in the Z direction!

the drag is calculated from the motion equations:

$m a_x = -Cd*A*rho(z)*|v|\cdot v_x$

we know everything here from the features, beside rho, which we calculate using to some empirical equation.
So we can bring the above equation to the form:

$\frac{Cd A}{m} = $ something we know

and the left hand term of the above equation is composed from all the stuff that characterize specific rockets (to my limited knowledge - maybe some rockets are identical on the outside and only differ in control?). That's why its a good feature.
## Results
We can see that we get OK results (0.58) with the relatively lousy classifier that we chose (lousy but it requires few seconds on a normal laptop! no GPUs and stuff). 
## To do next
1. **calculate the drage force for every time step, instead of the average for the whole trajectory.**
2. use a better classifier (fine tuned xgboost).
3. fit a simple polynomial to the original position and velocity in order to reduce noise in the accelerations, and use the better accelerations to calculate the drag coefficients.

In [1]:
import pandas as pd
import numpy as np
%matplotlib inline
import matplotlib.pyplot as plt

pd.options.display.max_columns = 200

# get data

In [2]:
df_train = pd.read_csv('data/train.csv/train.csv')
df_train = df_train.drop(['Unnamed: 0','targetName'], axis=1)
# df_test = pd.read_csv('data/test.csv/test.csv')



# some convenience functions

In [3]:
# give a list of specific column name at all given times
def cols(string, df):
    c = []
    for col in df.columns:
        if string in col:
            c.append(col)
    return c

# air density at height z, from http://www.dtic.mil/dtic/tr/fulltext/u2/a278141.pdf
def rho(z):
    rho_0 = 1.225 # Kg/m^3
    T0 = 288 # K
    Tz = T0 - 0.0065*z # K
    n = 5.2561
    return rho_0 * (Tz/T0)**n

# divide data to features and labels
def getXy(df):
    X = df.drop('class', axis=1).values
    y = df['class'].values
    return X, y

# add features to data

In [18]:
def treat(df):
    # make slicing easier
    posX = cols('posX', df)
    posZ = cols('posZ', df)
    posY = cols('posY', df)
    velY = cols('velY', df)
    velX = cols('velX', df)
    velZ = cols('velZ', df)
    # get rid of Y data
    df = df.drop(cols('Y', df), axis=1)
    # drop time data
    df = df.drop(cols('T', df), axis=1)
    # add skew and kurtosis (higher order moments / some statistical descriptions of the data...) 
    # I guess it gives information similar to the drag coefficient.
    df['pXskew'] = df[posX].skew(axis=1)
    df['pZskew'] = df[posZ].skew(axis=1)
    df['vXskew'] = df[velX].skew(axis=1)
    df['vZskew'] = df[velZ].skew(axis=1)
    df['pXkurt'] = df[posX].kurt(axis=1)
    df['pZkurt'] = df[posZ].kurt(axis=1)
    df['vXkurt'] = df[velX].kurt(axis=1)
    df['vZkurt'] = df[velZ].kurt(axis=1)
    # average position
    df['avgZ'] = np.nanmean(df[posZ], axis=1)
    df['avgX'] = np.nanmean(df[posX], axis=1)
    # average velocity
    df['avgVx'] = np.nanmean(df[velX], axis=1)
    df['avgVz'] = np.nanmean(df[velZ], axis=1)
    # average acceleration
    df['avgXacc'] = np.nanmean(df[velX].diff(axis=1), axis=1)
    df['avgZacc'] = np.nanmean(df[velX].diff(axis=1), axis=1)
    # average velocity size
    df['avgV2'] = df['avgVx']**2 + df['avgVz']**2
    df['avgV'] = np.sqrt(df['avgV2'])
    # derive drag force from the average values - NOT OPTIMAL
    df['dragX'] = -1.0 * df['avgXacc'] / (df['avgV'] * df['avgVx'] * rho(df['avgZ']))
    df['dragZ'] = -1.0 * ((df['avgXacc']-9.8) / (df['avgV'] * df['avgVz'] * rho(df['avgZ']))) 
    # average trajectory angle
    df['angleV'] = np.arctan(df['avgVz'] / df['avgVx'])
    # maybe the amount of samples give some insight? might be good for the competition but not for production.
    df['vecLen'] = np.sum(~np.isnan(df), axis=1)
    # replace v nan with 0
    df[cols('vel', df)] = df[cols('vel', df)].fillna(0, axis=1)
    df[cols('pos', df)] = df[cols('pos', df)].fillna(method='ffill', axis=1)
    # get rid of any nans that are still there
    df = df.fillna(0)
    
    return df

df_mod = treat(df_train)
df_mod.head()

,posX_0,posZ_0,velX_0,velZ_0,posX_1,posZ_1,velX_1,velZ_1,posX_2,posZ_2,velX_2,velZ_2,posX_3,posZ_3,velX_3,velZ_3,posX_4,posZ_4,velX_4,velZ_4,posX_5,posZ_5,velX_5,velZ_5,posX_6,posZ_6,velX_6,velZ_6,posX_7,posZ_7,velX_7,velZ_7,posX_8,posZ_8,velX_8,velZ_8,posX_9,posZ_9,velX_9,velZ_9,posX_10,posZ_10,velX_10,velZ_10,posX_11,posZ_11,velX_11,velZ_11,posX_12,posZ_12,velX_12,velZ_12,posX_13,posZ_13,velX_13,velZ_13,posX_14,posZ_14,velX_14,velZ_14,posX_15,posZ_15,velX_15,velZ_15,posX_16,posZ_16,velX_16,velZ_16,posX_17,posZ_17,velX_17,velZ_17,posX_18,posZ_18,velX_18,velZ_18,posX_19,posZ_19,velX_19,velZ_19,posX_20,posZ_20,velX_20,velZ_20,posX_21,posZ_21,velX_21,velZ_21,posX_22,posZ_22,velX_22,velZ_22,posX_23,posZ_23,velX_23,velZ_23,posX_24,posZ_24,velX_24,velZ_24,posX_25,posZ_25,velX_25,velZ_25,posX_26,posZ_26,velX_26,velZ_26,posX_27,posZ_27,velX_27,velZ_27,posX_28,posZ_28,velX_28,velZ_28,posX_29,posZ_29,velX_29,velZ_29,class,pXskew,pZskew,vXskew,vZskew,pXkurt,pZkurt,vXkurt,vZkurt,avgZ,avgX,avgVx,avgVz,avgXacc,avgZacc,avgV2,avgV,dragX,dragZ,angleV,vecLen
0,0.0,476.575673,486.926974,305.119216,241.974495,629.299199,483.225599,305.483552,478.641960,775.303988,479.884574,295.705799,722.250011,918.981911,476.162385,284.568090,958.719916,1055.371213,472.154455,272.206754,1190.880334,1196.262254,466.567232,268.333897,1420.132378,1321.228803,458.682625,259.327302,1652.390278,1450.869466,450.651136,256.915150,1880.680897,1573.532630,446.202964,243.623699,2102.864549,1700.786358,447.777201,240.314104,2323.274810,1812.297482,446.003514,224.761679,2542.011085,1925.256511,437.785557,225.442226,2758.315381,2038.966719,434.399012,215.780765,2038.966719,2038.966719,0.000000,0.000000,2038.966719,2038.966719,0.000000,0.000000,2038.966719,2038.966719,0.000000,0.000000,2038.966719,2038.966719,0.000000,0.000000,2038.966719,2038.966719,0.000000,0.000000,2038.966719,2038.966719,0.000000,0.000000,2038.966719,2038.966719,0.000000,0.000000,2038.966719,2038.966719,0.000000,0.000000,2038.966719,2038.966719,0.000000,0.000000,2038.966719,2038.966719,0.000000,0.000000,2038.966719,2038.966719,0.000000,0.000000,2038.966719,2038.966719,0.000000,0.000000,2038.966719,2038.966719,0.000000,0.000000,2038.966719,2038.966719,0.000000,0.000000,2038.966719,2038.966719,0.000000,0.000000,2038.966719,2038.966719,0.000000,0.000000,2038.966719,2038.966719,0.0,0.0,3,-0.053222,-0.141746,0.079598,0.067165,-1.201870,-1.174030,-1.519967,-1.217661,1298.056324,1405.548930,460.494095,261.352479,-4.377330,-4.377330,2.803599e+05,529.490254,0.000017,0.000098,0.516216,72
1,0.0,8073.292719,561.240579,16.840572,284.446308,8071.527369,555.568670,3.101450,554.676870,8074.335689,555.637391,5.754135,834.907832,8076.754846,550.590398,-1.185523,1107.203543,8075.488159,549.844356,-5.957418,1381.772797,8065.344592,553.250171,-12.404433,1660.681559,8061.326266,546.458364,-22.146437,1925.095650,8054.635525,542.319011,-23.550451,2208.707050,8041.410838,542.898667,-26.225190,2476.763027,8031.217211,538.531843,-31.075970,2750.552798,8009.619848,538.874990,-44.753241,3014.164407,7992.540553,536.919231,-38.623779,3287.218128,7970.448152,533.043208,-44.082259,3548.883625,7948.415734,539.939065,-46.791076,3810.090604,7919.189340,529.059177,-59.055140,4076.423555,7889.276698,527.531480,-62.382811,4343.713589,7860.394145,525.696016,-65.852992,4602.888498,7835.771317,524.479592,-69.462368,4873.860722,7795.721448,528.876211,-75.612622,5132.505343,7757.126646,525.369862,-86.204606,5389.360471,7717.650561,520.647418,-83.397893,5647.724523,7679.391661,513.142618,-83.089406,5903.463050,7630.018715,515.881276,-86.790140,6167.718859,7583.285984,511.135802,-98.629430,6425.017244,7539.828258,511.420694,-98.947146,6676.173904,7490.609771,509.701017,-108.548088,6927.863898,7435.299512,507.609961,-105.486628,7180.492375,7380.647332,504.388726,-114.297998,7380.647332,7380.647332,0.000000,0.000000,7380.647332,7380.647332,0.0,0.0,14,-0.038249,-0.802608,-0.012997,0.114401,-1.199082,-0.620766,-1.091926,-1.148153,7859.306032,3649.727508

# EDA
check the data for some patterns...

# train the "simplest", thin RF model, to see what we have here.
this will give us some measure of the feature importances, and generally will respond to enhancing the features quality which is what we want to know, fast.

In [19]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, log_loss


X, y = getXy(df_mod)

X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2)
print(X_train.shape, y_train.shape)

# Impute our data, then train
clf = RandomForestClassifier(n_estimators=50)
clf = clf.fit(X_train, y_train)

pred = clf.predict(X_val)
print(classification_report(y_pred=pred,y_true=y_val))
pred_proba = clf.predict_proba(X_val)
print('log_loss: {}'.format(log_loss(y_pred=pred_proba,y_true=y_val)))

(22996, 140) (22996,)
             precision    recall  f1-score   support

          1       0.98      0.98      0.98       318
          2       0.98      0.98      0.98       303
          3       0.61      0.73      0.66       345
          4       0.79      0.77      0.78       258
          5       0.51      0.52      0.52       298
          6       0.50      0.38      0.43       304
          7       0.63      0.73      0.68       317
          8       0.56      0.63      0.59       326
          9       0.41      0.28      0.33       277
         10       0.46      0.56      0.50       219
         11       0.72      0.67      0.70       172
         12       0.45      0.37      0.41       200
         13       0.71      0.77      0.74       263
         14       0.70      0.78      0.74       208
         15       0.44      0.46      0.45       160
         16       0.35      0.32      0.33       152
         17       0.69      0.66      0.67       207
         18       0.46 

# Feature importances
according to the random forest classifier.

We can see that the Xdrag is the leading feature, nice!
we can also see the X skew as second best. I think that the skew in X is related to the drag force...


In [20]:
feature_importance = pd.DataFrame({'value': clf.feature_importances_, 'feature': df_mod.drop('class', axis=1).columns})
feature_importance.sort_values('value', ascending=False).head(20)

,feature,value
136,dragX,0.063771
120,pXskew,0.039976
133,avgZacc,0.032365
132,avgXacc,0.031595
135,avgV,0.027911
134,avgV2,0.025489
124,pXkurt,0.012135
1,posZ_0,0.011607
9,posZ_2,0.011538
38,velX_9,0.011404
